# **页面基本设置**

In [2]:
cd /content/drive/My Drive/UTC&MTC/RUL-Net-master/XCC_File

/content/drive/My Drive/UTC&MTC/RUL-Net-master/XCC_File


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **加载文件并归一化**

In [ ]:
#数据加载
import pandas as pd
df_train = pd.read_csv("https://raw.githubusercontent.com/sivaji1233/09_turbofan_rul/master/data/train_FD001.txt", sep = ' ', header = None)

col_list = ['unit', 'time', 'os_1', 'os_2', 'os_3', 'sm_1', 'sm_2', 'sm_3', 'sm_4', 'sm_5', 'sm_6', 'sm_7', 'sm_8', 'sm_9', 'sm_10', 'sm_11', 'sm_12', 'sm_13', 'sm_14', 'sm_15', 'sm_16', 'sm_17', 'sm_18', 'sm_19', 'sm_20', 'sm_21']

df_train = df_train[list(range(26))]
df_train.columns = col_list
print(df_train.shape[0])
print(df_train.head())

# 数据可视化


# 去除无关变量
new_col_list = ['unit', 'time', 'os_1', 'os_2', 'sm_2', 'sm_3', 'sm_4', 'sm_6', 'sm_7', 'sm_8', 'sm_9', 'sm_11', 'sm_12', 'sm_13', 'sm_14', 'sm_15', 'sm_17', 'sm_20', 'sm_21']
df_train = df_train[new_col_list]
#df_test = df_test[new_col_list]


#归一化
def regularit(df):
    newDataFrame = pd.DataFrame(index=df.index)
    columns = df.columns.tolist()
    columns_0_4 = columns[:4]
    print(columns_0_4)
    for c in columns:
      if c in columns_0_4:
        print('yes')
        newDataFrame[c] = df[c]
        continue
      d = df[c]
      MAX = d.max()
      MIN = d.min()
      newDataFrame[c] = ((d - MIN) / (MAX - MIN)).tolist()
    return newDataFrame
df_train_0_1 = regularit(df_train)
print(df_train_0_1.shape[1],df_train_0_1.columns)
print(df_train_0_1.head())


# **结合样本均衡性要求和分类问题的类别整理汇总成新样本**


## *新样本组合（二分类）*

In [ ]:
####提取发生故障时的参数并单开一列做标记
number_fault = []
number_no_fault = []

df_train_xcc = df_train_0_1.copy()
df_train_xcc['fault?'] = 0  #新增列'fault?',发生故障标记为1，未发生故障标记为0
print(df_train_xcc.head())
 
for i in range(1,df_train_xcc.shape[0]):
  if df_train_xcc.loc[i,'time']<df_train_xcc.loc[i-1,'time']:
    number_fault.append(i-1)
    df_train_xcc.loc[i-1,'fault?'] = 1 
  else:
    number_no_fault.append(i)
number_fault.append(df_train_xcc.shape[0]-1)  #补最后一个发动机的故障数据
df_train_xcc.loc[df_train_xcc.shape[0]-1,'fault?'] = 1
df_fault = df_train_xcc.loc[number_fault,:]
print(len(number_fault))
df_no_fault = df_train_xcc.loc[number_no_fault,:]
print(len(number_no_fault))
 
df_train_xcc.head(195)
 
#导出故障时刻的汇总数据
#df_fault.to_csv('fault_state.csv')
 
 
###进行新样本组合：
'''  
通常将样本类别比例超过4：1或者3：1的数据看作是不均衡数据,
设置样本的均衡比例包括3：1、2：1、1：1、1：2，分别对应的样本量为400、300、200、150
设置样本的不均衡比例包括：4：1、10：1、20：1，分别对应的样本量为500、1100、2100
'''
import random
sample_type = {'eq':[400,300,200,150],'uneq':[500,1100,2100]}
sample_type['eq']
#随机生成均衡/不均衡样本
for j in sample_type.keys():
  for i in sample_type[j]:
    number_fault_temp = number_fault.copy()
    number_no_fault_300 = random.sample(number_no_fault, i-100)
    number_fault_temp.extend(number_no_fault_300)
    number_fault_temp.sort()   #得到序号并排列
    df_sample_temp = df_train_xcc.loc[number_fault_temp,:]  #得到对应行数的样本
 
    df_sample_temp.drop(columns=['unit','time'],inplace=True)
    print(df_sample_temp.head())
    df_sample_temp.to_csv('/content/drive/My Drive/UTC&MTC/RUL-Net-master/XCC_File/2_class/'+j+'/sample_'+str(i)+'.csv')
    print(len(number_fault_temp),'\n',number_fault_temp,'\n\n')

二分类算法执行

In [ ]:
!python D_R_E_S_2_class.py

## *新样本组合（多分类）*



In [ ]:
###进行新样本组合
'''
多分类的类别个数拟定，按照crnn的计算结果，拟定间隔为20，按照发生故障的时间递增地将类别标签设置为1，2，3，...。
标记0为无故障状态，故障样本的故障时间与标签严格对应，标签不一定序次连续。
设置样本的均衡比例分为几种：
①各类别标签的样本量相等（包括0）
②按照二分类的比例，将故障样本看成整体，故障样本内部类别比例不做改变。
'''
 
#查看故障样本的故障发生时间并做标记
fault_point = df_train_xcc.loc[number_fault,'time'].to_list()
print(min(fault_point),max(fault_point))
center_of_gap = [j for j in range(min(fault_point)+10,max(fault_point)-10,20)]
#class_fault = range(1,len(center_of_gap)+1)
#print(center_of_gap,'\n',class_fault)
fault_class = []
for i in fault_point:
  abs_error = [abs(j-i) for j in center_of_gap]
  index_fault = abs_error.index(min(abs_error)) + 1
  fault_class.append(index_fault)
print(fault_class)  
 
# 故障样本中类别统计
class_times = []
for i in range(1,12):
  counter = 0
  for j in fault_class:
    if j==i:
      counter = counter + 1
  class_times.append(counter)
  #print('class '+ str(i) + '元素个数为：' + counter )
  print(counter)
print(class_times)
 
'''
# 故障样本中类别统计条形图
import matplotlib.pyplot as plt
plt.xlabel('CLASS')
plt.ylabel('Number of Times')
plt.hist(class_times)
plt.show()
 
'''
#对故障&非故障数据重新标记  基于二分类已经标记过的数据
 
df_train_xcc_mc = df_train_xcc.copy()   # mc表示multi classification
number_fault_mc = number_fault.copy()
print(df_train_xcc_mc,'\n',number_fault_mc)
j = 0
for i in number_fault_mc:
  df_train_xcc_mc.loc[i,'fault?'] = fault_class[j]
  j = j+1
print(df_train_xcc_mc.head(2890))
 
#生成均衡样本（令无故障样本量 与 故障样本/故障类别=100/11=9 取值为9）
import random
number_no_fault_9 = random.sample(number_no_fault, 9)
number_fault_temp = number_fault.copy()
number_fault_temp.extend(number_no_fault_9)
number_fault_temp.sort()   #得到序号并排列
df_sample_temp = df_train_xcc_mc.loc[number_fault_temp,:]  #得到对应行数的样本
df_sample_temp.drop(columns=['unit','time'],inplace=True)  #删除'unit'和'time'列
print(df_sample_temp.head())
df_sample_temp.to_csv('/content/drive/My Drive/UTC&MTC/RUL-Net-master/XCC_File/multi_class/sample_eq.csv')
#print(len(number_fault_temp),'\n',number_fault_temp,'\n\n')
 
#生成非均衡样本（令无故障样本 与 故障样本呈二分类的比例
'''
设置样本的均衡比例包括3：1、2：1、1：1、1：2，分别对应的样本量为127、118、109、105
设置样本的不均衡比例包括：4：1、10：1、20：1，分别对应的样本量为136、190、280
'''
import random
sample_type = {'eq':[127,118,109,105],'uneq':[136,190,280]}
sample_type['eq']
#随机生成均衡/不均衡样本
for j in sample_type.keys():
  for i in sample_type[j]:
    number_fault_temp = number_fault.copy()
    number_no_fault_300 = random.sample(number_no_fault, i-100)
    number_fault_temp.extend(number_no_fault_300)
    number_fault_temp.sort()   #得到序号并排列
    df_sample_temp = df_train_xcc_mc.loc[number_fault_temp,:]  #得到对应行数的样本
 
    df_sample_temp.drop(columns=['unit','time'],inplace=True)
    print(df_sample_temp.head())
    df_sample_temp.to_csv('/content/drive/My Drive/UTC&MTC/RUL-Net-master/XCC_File/multi_class/'+j+'/sample_'+str(i)+'.csv')
    print(len(number_fault_temp),'\n',number_fault_temp,'\n\n')

多分类算法执行

In [ ]:
!python D_R_E_S_multi_class.py

# **按照C-MAPSS的训练和测试划分进行算法验证**

## 数据加载及分类样本生成

#### 数据加载、数据可视化、去除无关变量、归一化

In [ ]:
import pandas as pd
df_train = pd.read_csv("https://raw.githubusercontent.com/sivaji1233/09_turbofan_rul/master/data/train_FD003.txt", sep = ' ', header = None)
df_test = pd.read_csv("https://raw.githubusercontent.com/sivaji1233/09_turbofan_rul/master/data/test_FD003.txt", sep = ' ', header = None)

col_list = ['unit', 'time', 'os_1', 'os_2', 'os_3', 'sm_1', 'sm_2', 'sm_3', 'sm_4', 'sm_5', 'sm_6', 'sm_7', 'sm_8', 'sm_9', 'sm_10', 'sm_11', 'sm_12', 'sm_13', 'sm_14', 'sm_15', 'sm_16', 'sm_17', 'sm_18', 'sm_19', 'sm_20', 'sm_21']

df_train = df_train[list(range(26))]
df_train.columns = col_list

df_test = df_test[list(range(26))]
df_test.columns = col_list
#print(df_test.shape[0])
df_test.head()

# 数据可视化


# 去除无关变量
new_col_list = ['unit', 'time', 'os_1', 'os_2', 'sm_2', 'sm_3', 'sm_4', 'sm_6', 'sm_7', 'sm_8', 'sm_9', 'sm_11', 'sm_12', 'sm_13', 'sm_14', 'sm_15', 'sm_17', 'sm_20', 'sm_21']
df_train = df_train[new_col_list]
df_test = df_test[new_col_list]


# 归一化
def regularit(df):
    newDataFrame = pd.DataFrame(index=df.index)
    columns = df.columns.tolist()
    columns_0_4 = columns[:4]
    print(columns_0_4)
    for c in columns:
      if c in columns_0_4:
        print('yes')
        newDataFrame[c] = df[c]
        continue
      d = df[c]
      MAX = d.max()
      MIN = d.min()
      newDataFrame[c] = ((d - MIN) / (MAX - MIN)).tolist()
    return newDataFrame
df_train_normalized = regularit(df_train)
print(df_train_normalized.shape[0],df_train_normalized.columns)
#print(df_train_normalized.head())

df_test_normalized = regularit(df_test)
print(df_test_normalized.shape[0],df_test_normalized.columns)
#print(df_test_normalized.head())

#### 0-1标记（二分类样本生成）

In [ ]:
# 0-1标记
def mark_0_1(df):
  df_xcc = df.copy()
  number_fault = []
  number_no_fault = []
  df_xcc['fault?'] = 0  #新增列'fault?',发生故障标记为1，未发生故障标记为0
  #print(df_train_xcc.head())
  for i in range(1,df_xcc.shape[0]):
    if df_xcc.loc[i,'time']<df_xcc.loc[i-1,'time']:
      number_fault.append(i-1)
      df_xcc.loc[i-1,'fault?'] = 1 
    else:
      number_no_fault.append(i)
  number_fault.append(df_xcc.shape[0]-1)  #补最后一个发动机的故障数据
  df_xcc.loc[df_xcc.shape[0]-1,'fault?'] = 1
  df_fault = df_xcc.loc[number_fault,:]
  #print(len(number_fault))
  df_no_fault = df_xcc.loc[number_no_fault,:]
  #print(len(number_no_fault))
  number_sample = number_fault + number_no_fault
  number_sample.sort()
  
  #df_marked.head(195)
  return df_xcc, number_fault   #此处的df_trian_xcc是带了0-1标签的。

df_train_0_1_marked,number_fault_train = mark_0_1(df_train_normalized)
df_train_0_1_marked.to_csv('/content/drive/My Drive/UTC&MTC/RUL-Net-master/XCC_File/2_class_for_all_data/df_train_0_1_marked.csv')
#print(df_train_0_1_marked.shape)
#print(number_fault)
#print(df_train_0_1_marked.loc[number_fault_train,'fault?'])

df_test_0_1_marked,number_fault_test = mark_0_1(df_test_normalized)
df_test_0_1_marked.to_csv('/content/drive/My Drive/UTC&MTC/RUL-Net-master/XCC_File/2_class_for_all_data/df_test_0_1_marked.csv')
#print(df_test_0_1_marked.shape)
#print(df_test_0_1_marked.loc[number_fault_test,'fault?'])


print(number_fault_train)

'''
  # 生成均衡样本
  import random
  number_no_fault_9 = random.sample(number_no_fault, 9)
  number_fault_temp = number_fault.copy()
  number_fault_temp.extend(number_no_fault_9)
  number_fault_temp.sort()   #得到序号并排列
  df_sample_temp = df_train_xcc_mc.loc[number_fault_temp,:]  #得到对应行数的样本
  df_sample_temp.drop(columns=['unit','time'],inplace=True)  #删除'unit'和'time'列

'''

#### 1-11标记（多分类样本生成）

In [ ]:
#@title 默认标题文本
# 1-11标记
def mark_1_11(df_normalized):
  df_xcc, number_fault = mark_0_1(df_normalized)    # 继承用于二分类的已标记的样本
  df_xcc_mc = df_xcc.copy()
  #查看故障样本的故障发生时间并做标记
  fault_point = df_xcc.loc[number_fault,'time'].to_list()
  print(min(fault_point),max(fault_point))
  center_of_gap = [j for j in range(min(fault_point)+10,max(fault_point)-10,20)]
  #class_fault = range(1,len(center_of_gap)+1)
  #print(center_of_gap,'\n',class_fault)
  fault_class = []
  for i in fault_point:
    abs_error = [abs(j-i) for j in center_of_gap]
    index_fault = abs_error.index(min(abs_error)) + 1
    fault_class.append(index_fault)
  print(fault_class)  


  # 构建带标记的'fault?'列
  fault_col_mc = [0]*df_xcc.shape[0]
  for i in range(df_xcc.shape[0]):
    for j in number_fault:
      if i == j :
        fault_col_mc[i] = fault_class[number_fault.index(j)]
  print(fault_col_mc[1005])

  ## 方法一：删除'faul?'列，新增'fault_mc'列
  df_xcc_mc.drop(columns='fault?',inplace=True)
  df_xcc_mc['fault_mc'] = fault_col_mc
  print(df_xcc_mc.loc[number_fault[3],'fault_mc'])

  
  return df_xcc_mc

df_train_1_11_marked = mark_1_11(df_train_normalized)
df_train_1_11_marked.to_csv('/content/drive/My Drive/UTC&MTC/RUL-Net-master/XCC_File/multi_class_for_all_data/df_train_1_11_marked.csv')
#print(df_train_0_1_marked.shape)
#print(number_fault)
#print(df_train_0_1_marked.loc[number_fault_train,'fault?'])

df_test_1_11_marked = mark_1_11(df_test_normalized)
df_test_1_11_marked.to_csv('/content/drive/My Drive/UTC&MTC/RUL-Net-master/XCC_File/multi_class_for_all_data/df_test_1_11_marked.csv')
#print(df_test_0_1_marked.shape)
#print(df_test_0_1_marked.loc[number_fault_test,'fault?'])


## 应用D_R_E_S方法(二分类&多分类）

In [6]:
# 二分类
from D_R_E_S_2_class_for_all_data import data_load_xcc_original_train_test
from D_R_E_S_2_class_for_all_data import methods_classifier

X,y,X_,y_ = data_load_xcc_original_train_test(df_train_0_1_marked,df_test_0_1_marked)
print(X,'\n',y,'\n',X_,'\n',y_)

acc_1,acc_2,acc_3,acc_4 = methods_classifier(X,y,X_,y_)
print(acc_1,'\n',acc_2,'\n',acc_3,'\n',acc_4)

[[ 1.00000000e+00  1.00000000e+00 -5.00000000e-04 ...  2.72727273e-01
   5.59523810e-01  4.46330828e-01]
 [ 1.00000000e+00  2.00000000e+00  8.00000000e-04 ...  3.63636364e-01
   4.88095238e-01  5.34836256e-01]
 [ 1.00000000e+00  3.00000000e+00 -1.40000000e-03 ...  2.72727273e-01
   4.04761905e-01  4.58576862e-01]
 ...
 [ 1.00000000e+02  1.50000000e+02 -1.60000000e-03 ...  7.27272727e-01
   1.30952381e-01  1.81463958e-01]
 [ 1.00000000e+02  1.51000000e+02 -2.30000000e-03 ...  6.36363636e-01
   8.33333333e-02  1.88050840e-01]
 [ 1.00000000e+02  1.52000000e+02  0.00000000e+00 ...  7.27272727e-01
   2.32142857e-01  1.96771500e-01]] 
 [0 0 0 ... 0 0 1] 
 [[ 1.00000000e+00  1.00000000e+00 -5.00000000e-04 ...  2.72727273e-01
   5.59523810e-01  4.46330828e-01]
 [ 1.00000000e+00  2.00000000e+00  8.00000000e-04 ...  3.63636364e-01
   4.88095238e-01  5.34836256e-01]
 [ 1.00000000e+00  3.00000000e+00 -1.40000000e-03 ...  2.72727273e-01
   4.04761905e-01  4.58576862e-01]
 ...
 [ 1.00000000e+02  1.5

In [7]:
# 多分类
from D_R_E_S_multi_class_for_all_data import data_load_xcc_original_train_test
from D_R_E_S_multi_class_for_all_data import methods_classifier

X,y,X_,y_ = data_load_xcc_original_train_test(df_train_1_11_marked,df_test_1_11_marked)
print(X,'\n',y,'\n',X_,'\n',y_)

acc_1,acc_2,acc_3,acc_4 = methods_classifier(X,y,X_,y_)
print(acc_1,'\n',acc_2,'\n',acc_3,'\n',acc_4)

[[ 1.00000000e+00  1.00000000e+00 -5.00000000e-04 ...  2.72727273e-01
   5.59523810e-01  4.46330828e-01]
 [ 1.00000000e+00  2.00000000e+00  8.00000000e-04 ...  3.63636364e-01
   4.88095238e-01  5.34836256e-01]
 [ 1.00000000e+00  3.00000000e+00 -1.40000000e-03 ...  2.72727273e-01
   4.04761905e-01  4.58576862e-01]
 ...
 [ 1.00000000e+02  1.50000000e+02 -1.60000000e-03 ...  7.27272727e-01
   1.30952381e-01  1.81463958e-01]
 [ 1.00000000e+02  1.51000000e+02 -2.30000000e-03 ...  6.36363636e-01
   8.33333333e-02  1.88050840e-01]
 [ 1.00000000e+02  1.52000000e+02  0.00000000e+00 ...  7.27272727e-01
   2.32142857e-01  1.96771500e-01]] 
 [0 0 0 ... 0 0 1] 
 [[ 1.00000000e+00  1.00000000e+00 -5.00000000e-04 ...  2.72727273e-01
   5.59523810e-01  4.46330828e-01]
 [ 1.00000000e+00  2.00000000e+00  8.00000000e-04 ...  3.63636364e-01
   4.88095238e-01  5.34836256e-01]
 [ 1.00000000e+00  3.00000000e+00 -1.40000000e-03 ...  2.72727273e-01
   4.04761905e-01  4.58576862e-01]
 ...
 [ 1.00000000e+02  1.5